## AITA project: Cleaning the data.

In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.datasets import load_files
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.linear_model import LogisticRegression
import matplotlib.pyplot as plt
import re
%matplotlib inline
pd.set_option('display.max_colwidth', None)

Importing the database with comments:

In [2]:
data = pd.read_csv("aita_raw.csv")

In [3]:
data[20:30]

,body
20,YTA those were not your things. Let her eat what she wants to eat. Sure you had an “agreement” and it’s bullshit. If you don’t like having non vegan products in your household then don’t get roommates. That simple. You can’t control someone when they pay rent as well. You are a terrible human
21,NTA. He doesn’t get to decide what you do as a parent.
22,"NTA. They were told they had to abide by standards and rules of the home but used their age and status as parents as a means of controlling the household. They either could've been considerate, or hit the road."
23,"NTA- I almost didnt do professional for my own wedding but then was like... why the hell not. If I didn't and anyone did- it's because they had money in their budget for it.. and I didn't so who the hell cares. AND EVEN STILL I have friends that know how to professionally do their own make up- theirs still looked better than mine. No fucks given though. \n\n\nShe's ticked you're 22 and hot, and she had her venue at A MANSION but couldn't splurge on professional hair and make up to fit her own venue. pfffft \*rolls eyes\* \*lights fade\* \*curtains close\*"
24,"NTA. Tee doesn't get to decide Imani's emotions for her. She has been asked nicely to let the group know if China is going to be joining and has chosen to not do so and force China's presence on everyone, not just Imani. It appears that the group prefers Imani to China. Tee doesn't get to decide that China gets a pass when she has been neither invited nor was an original member of the group. She can think what she wants about Imani being ""babied"" or not; she was still an 'original' group member."
25,"NTA\n\nBut almost needing info becauuuuuuseeee \n\nHonestly as someone who has been almost 300 lbs before and down to 150 before, let me say these excuses that ""oh she's just insecure"" is zero excuse. Over inflation of ego does not equal ""I hate my body"". It seems like HAES really has taken ahold of people in that fat is good and you can be healthy. Unless you're a tall person, being over 200-250 probably will lead to health concerns down the road. I know from friends, not my experience, that all it took was one person going ""you know you're not (X) right?"". Regardless how it was said. If it was a shitty friend like mine (named kelly) who used to be 110 and now is 320 who pretends she's still 180ish it becomes an over inflated ego issue. She doesn't understand why her knees hurt. She's lost weight and is now 230 but for her height she is /still/ morbidly obese. \n\nThe bmi scale exists for a reason. These people who are morbidly obese even are recomended to just reduce calorie intake then when their body can take it then go into exercising and cico. \n\nTL;DR \n\nDepending on her height she could be morbodly obese and not know it. Fat is not, and will never be, healthy. There are health complications that are life threatening. If she's an ass and delusional she needs a wakeup call."
26,"YTA\n\nWow that escalated pretty quickly. You have NO clue what your biological parents OR your adopted parents went through when you got adopted. Your own spitefulness said its because of birth defect but for all you know, it could have been something completely different. For all you know, the situation could have been more complicated and the trauma from that was why your mom shut you down so quickly and your dad told you to focus on your adopted parents. \n\nYou demanded for things that you got told were best left buried. Yeah, it sucks. That's life. So your reaction was to destroy as many people as possible? If what you did blow up, it's going to affect not just your biological parents and your blood brother but also your adopted parents who do not seem to have done you any harm, along with who knows how many more of your family members who are completely innocent and not in the know. Do you realize how much damage you could be doing just because you decided to indulge in spitefulness?\n\nI really hope there's

In [4]:
data["body"]=data["body"].astype("str")

In [5]:
data.dtypes

body    object
dtype: object

We get rid of the punctuation and look for the the instances of YTA/NTA. YTA is labeled by 1, NTA by 0, mixed cases by 5.

In [6]:
def yta_nta(string):
    sentence = re.split("\W+",string.lower())
    x=0
    y=0
    if "yta" in sentence: x = 1
    if "nta" in sentence: y = 1
    if x==y: return 5
    elif x>y: return 1
    else: return 0

    

In [7]:
data["verdict"] = data["body"].apply(lambda x: yta_nta(x))

In [8]:
data.head()

,body,verdict
0,"I wanna say YTA since you could’ve approached this differently by asking her to stop coming in, but you’re 100% NTA if you have to ask that in the first place.",5
1,My thoughts exactly. It’s obviously NTA. Claire struggling with it doesn’t make her not accountable for her actions.,0
2,YTA. “In an ideal world” you wouldn’t have cheated and you wouldn’t be going behind her back trying to get confidential medical information. You’re not entitled to anything regarding the birth of the child she’s carrying.,1
3,"NTA, I’m a nude sleeper because as a teenager, being naked most of the time in my room was the only way people learned to knock and wait until I said it was okay.",0
4,YTA,1


We get rid of the punctuation and the YTA/NTA words.

In [9]:
def remove_nta_yta(string):
    sentence = " ".join([word for word in re.split("\W+",string) if word.lower() not in ["yta","nta"]])
    return sentence

In [10]:
data["body"] = data["body"].apply(lambda x: remove_nta_yta(x))

In [11]:
data.head()

,body,verdict
0,I wanna say since you could ve approached this differently by asking her to stop coming in but you re 100 if you have to ask that in the first place,5
1,My thoughts exactly It s obviously Claire struggling with it doesn t make her not accountable for her actions,0
2,In an ideal world you wouldn t have cheated and you wouldn t be going behind her back trying to get confidential medical information You re not entitled to anything regarding the birth of the child she s carrying,1
3,I m a nude sleeper because as a teenager being naked most of the time in my room was the only way people learned to knock and wait until I said it was okay,0
4,,1


In [12]:
data[data.verdict==5].count()

body       48586
verdict    48586
dtype: int64

Drop the mixed comments (both YTA and NTA in one comment): 

In [13]:
data = data[data.verdict != 5]

In [14]:
len(data)

3200451

In [15]:
data.tail()

,body,verdict
3249032,you put a fake obligation on yourself and used that to justify something,1
3249033,But I can t stress this enough TALK TO THEM It s not fair to disappear without telling them what s upm there is no way your relationship with them will be in the same place after this It will take you 15 minutes to type up a message saying you need some time to focus on yourself and you will be a little bit unavailable Tell them you still care for them but is important for your mental health If they are good friends they will understand,0
3249034,Fed is best no matter which way you do it Nobody should be shamed for being unable to breastfeed or for just not wanting to do it when formula is perfectly fine for getting the proper nutrition into the baby You do what works best for you and your baby,0
3249035,Negativity is very hard to be around I m glad you didn t just cuss him and break up You gave him solid advice He really might be depressed,0
3249036,Lmaoooo she s being a bad friend for talking behind the girls back You tried to address it nicely in the beginning and she couldn t get off her high horse not your fault S n she didn t even seem worried about the friends safety just mad she hooked up with someone,0


In [16]:
# data = data.drop(["body"], axis=1)

In [17]:
data.reset_index(drop=True, inplace=True)

In [18]:
print("number of YTA comments: {} \nnumber of NTA comments: {}".format( \
                                data[data.verdict==1].verdict.count(), data[data.verdict==0].verdict.count()) )

number of YTA comments: 1037563 
number of NTA comments: 2162888


In [19]:
2162888-1037563

1125325

Identifying the indices to drop:

In [20]:
np.random.seed(17)

remove_n = 1125325
drop_indices = np.random.choice(data[data.verdict==0].index, remove_n, replace=False)

In [21]:
data = data.drop(drop_indices)

In [22]:
print("number of YTA comments: {} \nnumber of NTA comments: {}".format( \
                                data[data.verdict==1].verdict.count(), data[data.verdict==0].verdict.count()) )

number of YTA comments: 1037563 
number of NTA comments: 1037563


In [23]:
data.head()

,body,verdict
0,My thoughts exactly It s obviously Claire struggling with it doesn t make her not accountable for her actions,0
1,In an ideal world you wouldn t have cheated and you wouldn t be going behind her back trying to get confidential medical information You re not entitled to anything regarding the birth of the child she s carrying,1
2,I m a nude sleeper because as a teenager being naked most of the time in my room was the only way people learned to knock and wait until I said it was okay,0
3,,1
4,The only thing worth considering in this post is 1 kid which pretty much negates your whole argument If it was just you and your wife let er rip I guess but having extremely dangerous surgery to prolong an older relative s life isn t putting your immediate family s best interests at heart If you die on that table everyone loses Why not ESH Although your wife phrased it in a shitty way she s probably just trying to stop you any way she can and she s right to do so,1


In [24]:
data = data[data.body != ""]

In [25]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2004709 entries, 0 to 3200449
Data columns (total 2 columns):
 #   Column   Dtype 
---  ------   ----- 
 0   body     object
 1   verdict  int64 
dtypes: int64(1), object(1)
memory usage: 45.9+ MB


In [26]:
data.head()

,body,verdict
0,My thoughts exactly It s obviously Claire struggling with it doesn t make her not accountable for her actions,0
1,In an ideal world you wouldn t have cheated and you wouldn t be going behind her back trying to get confidential medical information You re not entitled to anything regarding the birth of the child she s carrying,1
2,I m a nude sleeper because as a teenager being naked most of the time in my room was the only way people learned to knock and wait until I said it was okay,0
4,The only thing worth considering in this post is 1 kid which pretty much negates your whole argument If it was just you and your wife let er rip I guess but having extremely dangerous surgery to prolong an older relative s life isn t putting your immediate family s best interests at heart If you die on that table everyone loses Why not ESH Although your wife phrased it in a shitty way she s probably just trying to stop you any way she can and she s right to do so,1
7,As you said you don t care about the photo He asked you to remove it and you didn t just to be a jerk,1


In [ ]:
data.to_csv("aita_clean.csv")